## Sample Text

In [1]:
!cat phrases/phrase.txt

The forms of printed letters should be beautiful, and that their arrangement on the page should be reasonable and a help to the shapeliness of the letters.

# Tacotron2 + Waveglow

### PyTorch

In [2]:
!rm -rf __pycache__
!python inference.py -i phrases/phrase.txt --tacotron2 "checkpoints/nvidia_tacotron2pyt_fp16.pt" --waveglow "checkpoints/nvidia_waveglow256pyt_fp16.pt" --wn-channels 256 --sigma-infer 0.6 -o outputs/ --include-warmup --fp16

DLL 2022-10-18 04:04:31.071259 - PARAMETER input : phrases/phrase.txt 
DLL 2022-10-18 04:04:31.071358 - PARAMETER output : outputs/ 
DLL 2022-10-18 04:04:31.071383 - PARAMETER suffix :  
DLL 2022-10-18 04:04:31.071403 - PARAMETER tacotron2 : checkpoints/nvidia_tacotron2pyt_fp16.pt 
DLL 2022-10-18 04:04:31.071421 - PARAMETER waveglow : checkpoints/nvidia_waveglow256pyt_fp16.pt 
DLL 2022-10-18 04:04:31.071438 - PARAMETER parallelwavegan : None 
DLL 2022-10-18 04:04:31.071456 - PARAMETER sigma_infer : 0.6 
DLL 2022-10-18 04:04:31.071475 - PARAMETER denoising_strength : 0.01 
DLL 2022-10-18 04:04:31.071492 - PARAMETER sampling_rate : 22050 
DLL 2022-10-18 04:04:31.071509 - PARAMETER fp16 : True 
DLL 2022-10-18 04:04:31.071529 - PARAMETER cpu : False 
DLL 2022-10-18 04:04:31.071547 - PARAMETER log_file : nvlog.json 
DLL 2022-10-18 04:04:31.071563 - PARAMETER include_warmup : True 
DLL 2022-10-18 04:04:31.071580 - PARAMETER stft_hop_length : 256 
DLL 2022-10-18 04:04:31.071597 - PARAMETER mo

In [3]:
from IPython.display import Audio
display(Audio("outputs/waveglow_audio_0.wav"))

### TensorRT

In [7]:
!rm -rf __pycache__
!python inference_trt.py -i phrases/phrase.txt --encoder outputs/encoder_fp16.engine --decoder outputs/decoder_iter_fp16.engine --postnet outputs/postnet_fp16.engine --waveglow outputs/waveglow_fp16.engine -o outputs/ --include-warmup --fp16

[10/18/2022-00:56:28] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
[10/18/2022-00:56:29] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
====== Warm-up ======
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 1617 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running WaveGlow
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 1617 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running WaveGlow
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 1617 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running WaveGlow
====== Warm-up is done ======

Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 719 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running WaveGlow
DLL 2022-10-18 00:56:37.012354 - 0 tacotron2_encoder_latency : 0.008329784497618675 
DLL 2022-10-18 00:56:37.014237 - 0 tacotron2_decoder_latency : 0.

In [8]:
from IPython.display import Audio
display(Audio("outputs/waveglow_audio_0_trt.wav"))

### Latency Comparison

Run ten times per each inference case, and average the latencies

In [1]:
torch_latency_all = []
trt_latency_all = []
for i in range(10):
    !rm -rf __pycache__
    !python inference.py -i phrases/phrase.txt --tacotron2 "checkpoints/nvidia_tacotron2pyt_fp16.pt" --waveglow "checkpoints/nvidia_waveglow256pyt_fp16.pt" --wn-channels 256 --sigma-infer 0.6 -o outputs/ --include-warmup --fp16
    latency = !cat logs/nvlog.json | tail -1 | awk '{print $(NF)}' | sed 's/}//g'
    torch_latency_all = torch_latency_all + latency    
    
    !rm -rf __pycache__
    !python inference_trt.py -i phrases/phrase.txt --encoder outputs/encoder_fp16.engine --decoder outputs/decoder_iter_fp16.engine --postnet outputs/postnet_fp16.engine --waveglow outputs/waveglow_fp16.engine -o outputs/ --include-warmup --fp16
    latency = !cat logs/nvlog.json | tail -1 | awk '{print $(NF)}' | sed 's/}//g'
    trt_latency_all = trt_latency_all + latency
     

DLL 2022-10-18 01:12:00.793334 - PARAMETER input : phrases/phrase.txt 
DLL 2022-10-18 01:12:00.793424 - PARAMETER output : outputs/ 
DLL 2022-10-18 01:12:00.793448 - PARAMETER suffix :  
DLL 2022-10-18 01:12:00.793468 - PARAMETER tacotron2 : checkpoints/nvidia_tacotron2pyt_fp16.pt 
DLL 2022-10-18 01:12:00.793486 - PARAMETER waveglow : checkpoints/nvidia_waveglow256pyt_fp16.pt 
DLL 2022-10-18 01:12:00.793503 - PARAMETER parallelwavegan : None 
DLL 2022-10-18 01:12:00.793521 - PARAMETER sigma_infer : 0.6 
DLL 2022-10-18 01:12:00.793539 - PARAMETER denoising_strength : 0.01 
DLL 2022-10-18 01:12:00.793557 - PARAMETER sampling_rate : 22050 
DLL 2022-10-18 01:12:00.793574 - PARAMETER fp16 : True 
DLL 2022-10-18 01:12:00.793592 - PARAMETER cpu : False 
DLL 2022-10-18 01:12:00.793610 - PARAMETER log_file : nvlog.json 
DLL 2022-10-18 01:12:00.793627 - PARAMETER include_warmup : True 
DLL 2022-10-18 01:12:00.793643 - PARAMETER stft_hop_length : 256 
DLL 2022-10-18 01:12:00.793661 - PARAMETER mo

In [23]:
import numpy as np
torch_latency_all = [float(l) for l in torch_latency_all]
torch_latency_mean = np.array(torch_latency_all).mean()
torch_latency_std = np.array(torch_latency_all).std()
print("PyTorch")
print("avg latency:", torch_latency_mean)
print("latency std:", torch_latency_std)
print("")
trt_latency_all = [float(l) for l in trt_latency_all]
trt_latency_mean = np.array(trt_latency_all).mean()
trt_latency_std = np.array(trt_latency_all).std()
print("TensorRT")
print("avg latency:", trt_latency_mean)
print("latency std:", trt_latency_std)

PyTorch
avg latency: 1.2133198335766793
latency std: 0.03381237221362386

TensorRT
avg latency: 0.8103294470347464
latency std: 0.036496770134999065


# Tacotron2 + ParallelWaveGan

Some noises in the synthesis output due to the mel basis differnece between Tacotron2 and Parallelwavegan pretrained models.

### PyTorch

In [9]:
!rm -rf __pycache__
!python inference.py -i phrases/phrase.txt --tacotron2 "checkpoints/nvidia_tacotron2pyt_fp16.pt" --parallelwavegan "checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl" -o outputs/ --include-warmup --fp16

DLL 2022-10-18 00:35:27.165814 - PARAMETER input : phrases/phrase.txt 
DLL 2022-10-18 00:35:27.165895 - PARAMETER output : outputs/ 
DLL 2022-10-18 00:35:27.165920 - PARAMETER suffix :  
DLL 2022-10-18 00:35:27.165939 - PARAMETER tacotron2 : checkpoints/nvidia_tacotron2pyt_fp16.pt 
DLL 2022-10-18 00:35:27.165957 - PARAMETER waveglow : None 
DLL 2022-10-18 00:35:27.165975 - PARAMETER parallelwavegan : checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl 
DLL 2022-10-18 00:35:27.165992 - PARAMETER sigma_infer : 0.9 
DLL 2022-10-18 00:35:27.166010 - PARAMETER denoising_strength : 0.01 
DLL 2022-10-18 00:35:27.166027 - PARAMETER sampling_rate : 22050 
DLL 2022-10-18 00:35:27.166044 - PARAMETER fp16 : True 
DLL 2022-10-18 00:35:27.166066 - PARAMETER cpu : False 
DLL 2022-10-18 00:35:27.166083 - PARAMETER log_file : nvlog.json 
DLL 2022-10-18 00:35:27.166099 - PARAMETER include_warmup : True 
DLL 2022-10-18 00:35:27.166116 - PARAMETER stft_hop_length : 256 
DLL 2022-10-1

In [10]:
from IPython.display import Audio
display(Audio("outputs/parallelwavegan_audio_0.wav"))

### TensorRT

In [34]:
!rm -rf __pycache__
!python inference_trt.py -i phrases/phrase.txt --encoder outputs/encoder_fp16.engine --decoder outputs/decoder_iter_fp16.engine --postnet outputs/postnet_fp16.engine --parallelwavegan outputs/parallelwavegan_fp16.engine -o outputs/ --include-warmup --fp16

[10/18/2022-01:03:14] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
[10/18/2022-01:03:15] [TRT] [W] TensorRT was linked against cuDNN 8.4.1 but loaded cuDNN 8.4.0
====== Warm-up ======
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 315 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running ParallelWaveGan
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 315 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running ParallelWaveGan
Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 315 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running ParallelWaveGan
====== Warm-up is done ======

Running Tacotron2 Encoder
Running Tacotron2 Decoder
Stopping after 719 decoder steps
Running Tacotron2 Postnet
Tacotron2 Postnet done
Running ParallelWaveGan
DLL 2022-10-18 01:03:18.991544 - 0 tacotron2_encoder_latency : 0.009666560217738152 
DLL 2022-10-18 01:03:18.992007 - 0 tacot

In [13]:
from IPython.display import Audio
display(Audio("outputs/parallelwavegan_audio_0_trt.wav"))

### Latency Comparison

Run ten times per each inference case, and average the latencies

In [1]:
pwg_torch_latency_all = []
pwg_trt_latency_all = []
for i in range(10):
    !rm -rf __pycache__
    !python inference.py -i phrases/phrase.txt --tacotron2 "checkpoints/nvidia_tacotron2pyt_fp16.pt" --parallelwavegan "checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl" -o outputs/ --include-warmup --fp16
    latency = !cat logs/nvlog.json | tail -1 | awk '{print $(NF)}' | sed 's/}//g'
    pwg_torch_latency_all = pwg_torch_latency_all + latency    
    
    !rm -rf __pycache__
    !python inference_trt.py -i phrases/phrase.txt --encoder outputs/encoder_fp16.engine --decoder outputs/decoder_iter_fp16.engine --postnet outputs/postnet_fp16.engine --parallelwavegan outputs/parallelwavegan_fp16.engine -o outputs/ --include-warmup --fp16
    latency = !cat logs/nvlog.json | tail -1 | awk '{print $(NF)}' | sed 's/}//g'
    pwg_trt_latency_all = pwg_trt_latency_all + latency
     

DLL 2022-10-18 01:56:10.184186 - PARAMETER input : phrases/phrase.txt 
DLL 2022-10-18 01:56:10.184304 - PARAMETER output : outputs/ 
DLL 2022-10-18 01:56:10.184329 - PARAMETER suffix :  
DLL 2022-10-18 01:56:10.184348 - PARAMETER tacotron2 : checkpoints/nvidia_tacotron2pyt_fp16.pt 
DLL 2022-10-18 01:56:10.184366 - PARAMETER waveglow : None 
DLL 2022-10-18 01:56:10.184384 - PARAMETER parallelwavegan : checkpoints/ljspeech_parallel_wavegan.v1.long/checkpoint-1000000steps.pkl 
DLL 2022-10-18 01:56:10.184402 - PARAMETER sigma_infer : 0.9 
DLL 2022-10-18 01:56:10.184422 - PARAMETER denoising_strength : 0.01 
DLL 2022-10-18 01:56:10.184439 - PARAMETER sampling_rate : 22050 
DLL 2022-10-18 01:56:10.184457 - PARAMETER fp16 : True 
DLL 2022-10-18 01:56:10.184478 - PARAMETER cpu : False 
DLL 2022-10-18 01:56:10.184496 - PARAMETER log_file : nvlog.json 
DLL 2022-10-18 01:56:10.184513 - PARAMETER include_warmup : True 
DLL 2022-10-18 01:56:10.184529 - PARAMETER stft_hop_length : 256 
DLL 2022-10-1

In [2]:
import numpy as np
pwg_torch_latency_all = [float(l) for l in pwg_torch_latency_all]
pwg_torch_latency_mean = np.array(pwg_torch_latency_all).mean()
pwg_torch_latency_std = np.array(pwg_torch_latency_all).std()
print("PyTorch")
print("avg latency:", pwg_torch_latency_mean)
print("latency std:", pwg_torch_latency_std)
print("")
pwg_trt_latency_all = [float(l) for l in pwg_trt_latency_all]
pwg_trt_latency_mean = np.array(pwg_trt_latency_all).mean()
pwg_trt_latency_std = np.array(pwg_trt_latency_all).std()
print("TensorRT")
print("avg latency:", pwg_trt_latency_mean)
print("latency std:", pwg_trt_latency_std)

PyTorch
avg latency: 1.1515969078987838
latency std: 0.03195002763591988

TensorRT
avg latency: 0.7865189269185067
latency std: 0.03747897671252636
